In [65]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt


In [66]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'usingdevice: { device}')

usingdevice: cpu


In [67]:
torch.manual_seed(42)

In [68]:
df =pd.read_csv('fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [69]:
df.shape

(6000, 785)

In [70]:
x=df.iloc[:,1:].values
y=df.iloc[:,0].values


In [71]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [72]:
x_train=x_train/255.0
x_test=x_test/255.0

In [73]:
x_train

array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.0627451, 0.       ,
        0.       ]])

In [74]:
#create customeDataset Class
class CustomDataset(Dataset):
    def __init__(self,features,labels):
      self.features=torch.tensor(features,dtype=torch.float32)
      self.labels=torch.tensor(labels,dtype=torch.long)

    def  __len__(self):
      return len(self.features)

    def __getitem__(self,item):
      return self.features[item],self.labels[item]



In [75]:
# Train dataset object create
train_dataset=CustomDataset(x_train,y_train)
len(train_dataset)


4800

In [76]:
test_dataset=CustomDataset(x_test,y_test)
len(test_dataset)

1200

In [77]:
# train_test_loader create
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True,pin_memory=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False,pin_memory=True)

In [78]:
len(train_loader)

150

In [79]:
class MyNN(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(num_features,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)
    )

  def forward(self, x):  # Properly aligned at class level
    return self.model(x)


In [80]:
epochs=100
learning_rate=0.1

In [81]:
#instantiate the model
model=MyNN(x_train.shape[1]) #show data in features

#model for Gpu
model=model.to(device)


#loss function
criterion=nn.CrossEntropyLoss()

#optimizer
optimizer=optim.SGD(model.parameters(),lr=learning_rate)



In [84]:
# training_loop
for epoch in range(epochs):
    total_epoch_loss=0
    for batch_features, batch_labels in train_loader:

        #For GPU
        batch_feature,batch_labels=batch_features.to_(device),batch_labels.to_(device)

        # forward pass
        outputs = model(batch_features)


        # calculate loss
        loss = criterion(outputs, batch_labels)

        # backward pass
        optimizer.zero_grad()
        loss.backward()

        # update gradients
        optimizer.step()
        total_epoch_loss=total_epoch_loss+loss.item()
    avg_loss=total_epoch_loss/len(train_loader)
    print(f'epoch:{epoch+1}, loss: {avg_loss}')

epoch:1, loss: 1.313965723514557
epoch:2, loss: 0.7639605889717738
epoch:3, loss: 0.6432088449597358
epoch:4, loss: 0.5827425147096316
epoch:5, loss: 0.532613729039828
epoch:6, loss: 0.49239992529153825
epoch:7, loss: 0.4699807928005854
epoch:8, loss: 0.44091923266649247
epoch:9, loss: 0.424676075776418
epoch:10, loss: 0.4080882055560748
epoch:11, loss: 0.3834374950329463
epoch:12, loss: 0.37474448124567666
epoch:13, loss: 0.34125059788425766
epoch:14, loss: 0.3368989329040051
epoch:15, loss: 0.3219494473437468
epoch:16, loss: 0.30843092729647953
epoch:17, loss: 0.29012444655100506
epoch:18, loss: 0.28416174963116647
epoch:19, loss: 0.2766579023996989
epoch:20, loss: 0.2607216949760914
epoch:21, loss: 0.25234069575866064
epoch:22, loss: 0.23786049174765747
epoch:23, loss: 0.23384537016352017
epoch:24, loss: 0.22713854680458703
epoch:25, loss: 0.2278678063551585
epoch:26, loss: 0.2135811729480823
epoch:27, loss: 0.19571318417787553
epoch:28, loss: 0.19503607782224813
epoch:29, loss: 0.2

In [85]:
# model evaluation perpose
model.eval()


MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [87]:
# eval code
total=0
correct=0
with torch.no_grad():
  for batch_features,batch_labels in test_loader:
    #For GPU
    batch_feature,batch_labels=batch_features.to_(device),batch_labels.to_(device)
    outputs=model(batch_features)

    _,predicted=torch.max(outputs,1)

    total=total+batch_labels.shape[0]
    correct=correct+(predicted==batch_labels).sum().item()
print(correct/total)

0.8366666666666667


In [ ]:
# improve accuracy value
# use full dataset(large date set use gpu)
# optimizer,lr,epochs,weight_int
# hpt(hyper parameter training)